In [ ]:
!pip install tensorflow_io
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas
import tensorflow_datasets as tfds
import time
import librosa.display as lidp
import tensorflow_io as tfio
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Conv1DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.constraints import Constraint
from numpy import expand_dims
from numpy import mean
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from IPython import display

In [ ]:
import json

In [ ]:
data, info = tfds.load('nsynth', try_gcs=True, split='train', with_info=True)
assert isinstance(data, tf.data.Dataset)
#get data

In [ ]:
testdata, info = tfds.load('nsynth', try_gcs=True, split='test', with_info=True)
assert isinstance(testdata, tf.data.Dataset)
print(len(testdata))
test_x = np.array([])
test_y = np.array([])
count = 0
for i in testdata.take(4096):
	# select images
  sound = i["audio"]
  sound = sound[0:16384].numpy()
  test_x = np.append(test_x,sound) 
  # generate class labels, -1 for 'real'
  label = i["instrument"]["family"]
  c = label.numpy()
  #print(c)
  reallabel = np.zeros([11])
  reallabel[c] = 1
  test_y = np.append(test_y,reallabel)
  print(count)
  count += 1
print(test_x)
print(test_y)

In [ ]:
test_x = np.reshape(test_x,(4096,16384,1))
test_y = np.reshape(test_y,(4096,11))
print(test_x.shape)
print(test_y.shape)

In [ ]:
# image size
sound_size = 16384
channels = 1
sound_shape = (sound_size, channels)    # (16384,1)

batch_size = 256

In [ ]:
dataset = data.shuffle(batch_size*16).batch(batch_size)
total = len(dataset)
dataset = data.shuffle(batch_size*16).batch(batch_size).repeat()
db_iter = iter(dataset)

In [ ]:
for i in range(1):
  batch = next(db_iter)
  label = batch["instrument"]["family"]
  c = label.numpy()
  print(c)
  reallabel = np.zeros([label.shape[0],11])
  reallabel[np.arange(reallabel.shape[0]),c] = 1
  print(reallabel)
  output = tf.convert_to_tensor(reallabel)
  print(output)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#login your dirve so that you can save model data for later training

In [ ]:
def conv_block(
    x,
    filters,
    activation,
    kernel_size=25,
    strides=4,
    padding="same",
    use_bias=True,
    use_bn=False,
    use_dropout=False,
    drop_value=0.5,
):
    x = layers.Conv1D(
        filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias
    )(x)
    if use_bn:
        x = layers.BatchNormalization()(x)
    x = activation(x)
    if use_dropout:
        x = layers.Dropout(drop_value)(x)
    return x

In [ ]:
def get_discriminator_model(d,c):
    input = layers.Input(shape=sound_shape)
    x = conv_block(
        input,
        d,
        kernel_size=25,
        strides=4,
        use_bn=True,
        use_bias=True,
        activation=layers.LeakyReLU(0.2),
        use_dropout=False,
        drop_value=0.3,
    )
    x = layers.MaxPooling1D(pool_size=4,strides=1,padding='same')(x)
    x = conv_block(
        x,
        2*d,
        kernel_size=25,
        strides=4,
        use_bn=True,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=False,
        drop_value=0.3,
    )
    x = layers.MaxPooling1D(pool_size=4,strides=1,padding='same')(x)
    x = conv_block(
        x,
        4*d,
        kernel_size=25,
        strides=4,
        use_bn=True,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=False,
        drop_value=0.3,
    )
    x = layers.MaxPooling1D(pool_size=4,strides=1,padding='same')(x)
    x = conv_block(
        x,
        8*d,
        kernel_size=25,
        strides=4,
        use_bn=True,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=False,
        drop_value=0.3,
    )
    x = layers.MaxPooling1D(pool_size=4,strides=1,padding='same')(x)
    x = conv_block(
        x,
        16*d,
        kernel_size=25,
        strides=4,
        use_bn=True,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=False,
        drop_value=0.3,
    )
    x = layers.MaxPooling1D(pool_size=4,strides=1,padding='same')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(11,activation='softmax')(x)

    d_model = keras.models.Model(input, x, name="discriminator")
    return d_model

In [ ]:
d_model = get_discriminator_model(64,1)
d_model.summary()

In [ ]:
def plot_history(d_hist, epoch):
  # plot history
  plt.plot(d_hist, label='crit')
  plt.legend()
  plt.savefig('/content/gdrive/My Drive/IS/plot_line_plot_loss_{:04d}.png'.format(epoch))
  plt.close()
  with open("/content/gdrive/My Drive/IS/derror.txt", "w") as fp:  
    json.dump(d_hist, fp)

In [ ]:
d_hist, g_hist = list(), list()

In [ ]:
with open("/content/gdrive/My Drive/IS/derror.txt", "r") as fp:  
  d_hist = json.load(fp)

In [ ]:
discriminator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)

In [ ]:
d_model.compile(optimizer=discriminator_optimizer,loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy']);

In [ ]:
def generate_real_samples(dataset, n_samples):
	# choose random instances
	batch = next(dataset)
	# select images
	sound = batch["audio"]
	sound = sound[:,0:16384]
	X = tf.reshape(sound,[sound.shape[0],16384,1])
	# generate class labels, -1 for 'real'
	label = batch["instrument"]["family"]
	c = label.numpy()
	reallabel = np.zeros([label.shape[0],11])
	reallabel[np.arange(reallabel.shape[0]),c] = 1
	#print(reallabel)
	y = tf.convert_to_tensor(reallabel)
	return X, y

In [ ]:
# train the critic
def train(d_model, dataset,test_x,test_y, n_epochs=5, n_batch=64):
	for i in range(n_epochs):
		# get randomly selected 'real' samples
		X_real, y_real = generate_real_samples(dataset, n_batch)
		# update critic model weights
		d_loss = d_model.train_on_batch(X_real, y_real)
		d_hist.append(d_loss[0])
		# summarize loss on this batch
		print('>%d, d=%.3f' % (i+1, d_loss[0]))
		# evaluate the model performance every 'epoch'
		if (i+1) % (4000) == 0:
			checkpoint.save(file_prefix = checkpoint_prefix)
			plot_history(d_hist, len(d_hist))
			d_model.evaluate(test_x,test_y)

In [ ]:
checkpoint_dir = '/content/gdrive/My Drive/IS'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(d_model=d_model)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
checkpoint.save(file_prefix = checkpoint_prefix)

In [ ]:
plot_history(d_hist,-1)

In [ ]:
d_model.evaluate(test_x,test_y)

In [ ]:
d_model.save("/content/gdrive/My Drive/IS/rmodel");

In [ ]:
model = keras.models.load_model('/content/gdrive/My Drive/IS/rmodel')

In [ ]:
train(d_model,db_iter,test_x,test_y,50001)